<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Exploring-data" data-toc-modified-id="Exploring-data-1">Exploring data</a></span><ul class="toc-item"><li><span><a href="#Downloading-files" data-toc-modified-id="Downloading-files-1.1">Downloading files</a></span></li><li><span><a href="#Check-for-duplicates" data-toc-modified-id="Check-for-duplicates-1.2">Check for duplicates</a></span></li><li><span><a href="#gene_id-column" data-toc-modified-id="gene_id-column-1.3">gene_id column</a></span><ul class="toc-item"><li><span><a href="#Text-mining" data-toc-modified-id="Text-mining-1.3.1">Text-mining</a></span></li><li><span><a href="#Knowledge" data-toc-modified-id="Knowledge-1.3.2">Knowledge</a></span></li><li><span><a href="#Experiments" data-toc-modified-id="Experiments-1.3.3">Experiments</a></span></li></ul></li><li><span><a href="#gene_name-column" data-toc-modified-id="gene_name-column-1.4">gene_name column</a></span></li><li><span><a href="#disease_id" data-toc-modified-id="disease_id-1.5">disease_id</a></span><ul class="toc-item"><li><span><a href="#Text-mining" data-toc-modified-id="Text-mining-1.5.1">Text-mining</a></span></li><li><span><a href="#Knowledge" data-toc-modified-id="Knowledge-1.5.2">Knowledge</a></span></li><li><span><a href="#Experiments" data-toc-modified-id="Experiments-1.5.3">Experiments</a></span></li></ul></li><li><span><a href="#Other-columns" data-toc-modified-id="Other-columns-1.6">Other columns</a></span><ul class="toc-item"><li><span><a href="#Text-mining" data-toc-modified-id="Text-mining-1.6.1">Text-mining</a></span></li><li><span><a href="#Knowledge" data-toc-modified-id="Knowledge-1.6.2">Knowledge</a></span></li><li><span><a href="#Experiments" data-toc-modified-id="Experiments-1.6.3">Experiments</a></span></li></ul></li></ul></li><li><span><a href="#Remove-rows" data-toc-modified-id="Remove-rows-2">Remove rows</a></span></li><li><span><a href="#Pre-NodeNorming" data-toc-modified-id="Pre-NodeNorming-3">Pre-NodeNorming</a></span><ul class="toc-item"><li><span><a href="#Genes" data-toc-modified-id="Genes-3.1">Genes</a></span></li><li><span><a href="#Diseases" data-toc-modified-id="Diseases-3.2">Diseases</a></span></li><li><span><a href="#Adding-NodeNorm-data,-removing-rows" data-toc-modified-id="Adding-NodeNorm-data,-removing-rows-3.3">Adding NodeNorm data, removing rows</a></span></li><li><span><a href="#Exploring-&quot;duplicates&quot;-from-NodeNorming" data-toc-modified-id="Exploring-&quot;duplicates&quot;-from-NodeNorming-3.4">Exploring "duplicates" from NodeNorming</a></span></li></ul></li><li><span><a href="#Generating-documents" data-toc-modified-id="Generating-documents-4">Generating documents</a></span><ul class="toc-item"><li><span><a href="#Rows-not-included" data-toc-modified-id="Rows-not-included-4.1">Rows not included</a></span></li><li><span><a href="#Columns-not-included" data-toc-modified-id="Columns-not-included-4.2">Columns not included</a></span></li><li><span><a href="#File:-List-of-TRAPI-edges" data-toc-modified-id="File:-List-of-TRAPI-edges-4.3">File: List of TRAPI edges</a></span></li><li><span><a href="#File:-KGX-edges" data-toc-modified-id="File:-KGX-edges-4.4">File: KGX edges</a></span></li><li><span><a href="#File:-KGX-nodes" data-toc-modified-id="File:-KGX-nodes-4.5">File: KGX nodes</a></span></li></ul></li><li><span><a href="#Notes" data-toc-modified-id="Notes-5">Notes</a></span></li></ul></div>

# Notebook for DISEASES parser development

In [1]:
## not for parser. for notebook only 

## CX: allows multiple lines of code to print from one code block
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Exploring data

### Downloading files

Going to load entire files for review. 

In parser, may want to do a generator approach and ingest large chunks (ex: 1000-2000 lines) at 1 time. This balances "less memory footprint" vs "NodeNorming in large batches is faster". 

(pandas [read_table](https://pandas.pydata.org/docs/reference/api/pandas.read_table.html) has an iterator for rows/chunks! see iterator/chunksize parameters)

In [2]:
## put in parser (format)
import pathlib
import pandas as pd

## don't put in parser. Just for this notebook
from pprint import pprint

## unsure on putting into parser: more for notebook viewing/debugging...
pd.options.display.max_columns = None

In [3]:
## useful function for exploring data
def check_if_contains(df, column_name, patterns):
    for i in patterns:
        temp = df[df[column_name].str.contains(pat=i, case=False)]
        if temp.size > 0:
            print(f'"{i}"')
            print(temp.shape)

<div class="alert alert-block alert-danger">
    
This notebook was originally written using data files downloaded 2025-04-14 between 4:23-4:24 PM Pacific time (23:23-23:24 UTC+0) from https://diseases.jensenlab.org/Downloads. 

In [4]:
## put in parser (format)

## paths to raw data files

base_file_path = pathlib.Path.home().joinpath("Desktop", "DISEASES_files", "2025_07_17")

textmining_path = base_file_path.joinpath("human_disease_textmining_filtered.tsv")
knowledge_path = base_file_path.joinpath("human_disease_knowledge_filtered.tsv")
experiments_path = base_file_path.joinpath("human_disease_experiments_filtered.tsv")


textmining_path
knowledge_path
experiments_path

PosixPath('/Users/colleenxu/Desktop/DISEASES_files/2025_07_17/human_disease_textmining_filtered.tsv')

PosixPath('/Users/colleenxu/Desktop/DISEASES_files/2025_07_17/human_disease_knowledge_filtered.tsv')

PosixPath('/Users/colleenxu/Desktop/DISEASES_files/2025_07_17/human_disease_experiments_filtered.tsv')

In [5]:
## put in parser (format)

## download files

## files have no headers: adding based on https://diseases.jensenlab.org/Downloads
textmining_header = ["gene_id", "gene_name", "disease_id", "disease_name", 
                     "z_score", "confidence_score", "url"]
knowledge_header = ["gene_id", "gene_name", "disease_id", "disease_name", 
                    "source_db", "evidence_type", "confidence_score"]


df_textmining = pd.read_table(textmining_path, sep="\t", names=textmining_header)
df_knowledge = pd.read_table(knowledge_path, sep="\t", names=knowledge_header)

In [6]:
## loading experiments: commenting out experiments exploration just in case

# experiments_header = ["gene_id", "gene_name", "disease_id", "disease_name", 
#                     "source_db", "source_score", "confidence_score"]


# df_experiments = pd.read_table(experiments_path, sep="\t", names=experiments_header)

<div class="alert alert-block alert-info">

No missing values

In [6]:
df_textmining.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288901 entries, 0 to 288900
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   gene_id           288901 non-null  object 
 1   gene_name         288901 non-null  object 
 2   disease_id        288901 non-null  object 
 3   disease_name      288901 non-null  object 
 4   z_score           288901 non-null  float64
 5   confidence_score  288901 non-null  float64
 6   url               288901 non-null  object 
dtypes: float64(2), object(5)
memory usage: 113.8 MB


In [7]:
df_textmining

,gene_id,gene_name,disease_id,disease_name,z_score,confidence_score,url
0,18S_rRNA,18S_rRNA,DOID:9643,Babesiosis,7.245,3.623,https://diseases.jensenlab.org/Entity?document...
1,18S_rRNA,18S_rRNA,DOID:3733,Theileriasis,6.371,3.185,https://diseases.jensenlab.org/Entity?document...
2,18S_rRNA,18S_rRNA,DOID:12365,Malaria,6.299,3.150,https://diseases.jensenlab.org/Entity?document...
3,18S_rRNA,18S_rRNA,DOID:9640,Sarcocystosis,6.142,3.071,https://diseases.jensenlab.org/Entity?document...
4,18S_rRNA,18S_rRNA,DOID:1733,Cryptosporidiosis,6.119,3.059,https://diseases.jensenlab.org/Entity?document...
...,...,...,...,...,...,...,...
288896,snoU13,snoU13,DOID:0110084,Arrhythmogenic right ventricular dysplasia 13,3.775,1.887,https://diseases.jensenlab.org/Entity?document...
288897,snoU13,snoU13,DOID:0110408,Retinitis pigmentosa 11,3.387,1.693,https://diseases.jensenlab.org/Entity?document...
288898,snoU13,snoU13,DOID:1849,Cannabis dependence,3.228,1.614,https://diseases.jensenlab.org/Entity?document...
288899,snoU13,snoU13,DOID:0060775,Microvillus inclusion disease,3.147,1.573,https://diseases.jensenlab.org/Entity?document...


In [8]:
df_knowledge.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7770 entries, 0 to 7769
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   gene_id           7770 non-null   object
 1   gene_name         7770 non-null   object
 2   disease_id        7770 non-null   object
 3   disease_name      7770 non-null   object
 4   source_db         7770 non-null   object
 5   evidence_type     7770 non-null   object
 6   confidence_score  7770 non-null   int64 
dtypes: int64(1), object(6)
memory usage: 2.8 MB


In [9]:
df_knowledge

,gene_id,gene_name,disease_id,disease_name,source_db,evidence_type,confidence_score
0,ABHD11-AS1,ABHD11-AS1,DOID:1928,Williams-Beuren syndrome,MedlinePlus,CURATED,5
1,ENSP00000001146,CYP26B1,DOID:2340,Craniosynostosis,UniProtKB-KW,CURATED,4
2,ENSP00000003084,CFTR,DOID:0111862,Congenital bilateral absence of vas deferens,MedlinePlus,CURATED,5
3,ENSP00000003084,CFTR,DOID:1485,Cystic fibrosis,MedlinePlus,CURATED,5
4,ENSP00000005226,USH1C,DOID:0050439,Usher syndrome,MedlinePlus,CURATED,5
...,...,...,...,...,...,...,...
7765,hsa-miR-145-5p,hsa-miR-145-5p,DOID:0090016,Chromosome 5q deletion syndrome,MedlinePlus,CURATED,5
7766,hsa-miR-146a-5p,hsa-miR-146a-5p,DOID:0090016,Chromosome 5q deletion syndrome,MedlinePlus,CURATED,5
7767,hsa-miR-184,hsa-miR-184,DOID:10126,Keratoconus,MedlinePlus,CURATED,5
7768,hsa-miR-590-5p,hsa-miR-590-5p,DOID:1928,Williams-Beuren syndrome,MedlinePlus,CURATED,5


### Check for duplicates

<div class="alert alert-block alert-info">

Duplicates found in knowledge file. This is a DISEASES-side problem.
    
Ideas for how to handle duplicates (including if they show up in text-mining file later): 
* ingest entire file and remove duplicates
* **doing this right now**: check a set of "already-done" edges (gene_id, disease_id, source_db for knowledge) right before creating document (earlier steps will cut down number of rows to create documents with). 

In [10]:
df_textmining[df_textmining.duplicated(subset=["gene_id", "disease_id"], keep=False)].shape

(0, 7)

In [11]:
df_knowledge[df_knowledge.duplicated(keep=False)].shape

(16, 7)

In [12]:
## double-checking if specific column subset can work to check for duplicates
df_knowledge[df_knowledge.duplicated(subset=["gene_id", "disease_id", "source_db"], keep=False)].shape
## same count so it can

## print for manual review
df_knowledge[df_knowledge.duplicated(keep=False)]

(16, 7)

,gene_id,gene_name,disease_id,disease_name,source_db,evidence_type,confidence_score
1617,ENSP00000269703,CYP4F22,DOID:0060655,Autosomal recessive congenital ichthyosis,MedlinePlus,CURATED,5
1618,ENSP00000269703,CYP4F22,DOID:0060655,Autosomal recessive congenital ichthyosis,MedlinePlus,CURATED,5
1667,ENSP00000272895,ABCA12,DOID:0060655,Autosomal recessive congenital ichthyosis,MedlinePlus,CURATED,5
1668,ENSP00000272895,ABCA12,DOID:0060655,Autosomal recessive congenital ichthyosis,MedlinePlus,CURATED,5
1992,ENSP00000291295,CALM3,DOID:2843,Long QT syndrome,UniProtKB-KW,CURATED,4
1993,ENSP00000291295,CALM3,DOID:2843,Long QT syndrome,UniProtKB-KW,CURATED,4
2577,ENSP00000311687,NIPAL4,DOID:0060655,Autosomal recessive congenital ichthyosis,MedlinePlus,CURATED,5
2578,ENSP00000311687,NIPAL4,DOID:0060655,Autosomal recessive congenital ichthyosis,MedlinePlus,CURATED,5
3433,ENSP00000342392,MESP2,DOID:0050568,Spondylocostal dysostosis,MedlinePlus,CURATED,5
3434,ENSP00000342392,MESP2,DOID:0050568,Spondylocostal dysostosis,MedlinePlus,CURATED,5


In [14]:
# df_experiments[df_experiments.duplicated(keep=False)].shape

### gene_id column

<div class="alert alert-block alert-info">

**A small portion of the data doesn't have ENSP IDs (text-mining and knowledge)!** Instead, the values are names (same as gene_name). **Drop these rows from further parsing (can't NodeNorm)** 
    
I checked this "no ENSP" data for other ID namespaces mentioned in the paper, but couldn't find any matches. Other ID namespaces mentioned in [paper's Materials and methods](https://academic.oup.com/database/article/doi/10.1093/database/baac019/6554833?login=false#344427091) -> Dictionaries section:

> The human gene dictionary was obtained from STRING v11.0 (27) and is based on information from **Ensembl (34), UniProtKB (12) and HGNC (35)** databases.

In [13]:
## check for delimiters (between prefix and ID, between IDs)

delimiters = [",", ";", ":"]

print("text-mining")
check_if_contains(df_textmining, "gene_id", delimiters)

print("knowledge")
check_if_contains(df_knowledge, "gene_id", delimiters)

## didn't print any stats, so not found

text-mining
knowledge


In [14]:
## double-check work on 1 value
df_textmining[df_textmining["gene_id"].str.contains(":", case=False)]

,gene_id,gene_name,disease_id,disease_name,z_score,confidence_score,url


In [17]:
## looked at "_": sometimes it can be a delimiter between prefix and ID
## text-mining: there are values, but all were names

# df_textmining[df_textmining["gene_id"].str.contains("_", case=False)]["gene_id"].unique()

# len(df_textmining[df_textmining["gene_id"].str.contains("_", case=False)]["gene_id"].unique())

In [15]:
## no values

df_knowledge[df_knowledge["gene_id"].str.contains("_")].shape

(0, 7)

#### Text-mining

In [16]:
## how much data doesn't have ENSP ID
df_no_ENSP_textmining = df_textmining[~ df_textmining["gene_id"].str.startswith("ENSP")].copy()
df_no_ENSP_textmining.shape

## vs entire dataset
df_no_ENSP_textmining.shape[0] / df_textmining.shape[0]

(16541, 7)

0.05725490739042094

In [17]:
## when data doesn't have ENSP ID, are ID == name?
df_no_ENSP_textmining[df_no_ENSP_textmining["gene_id"] == 
                      df_no_ENSP_textmining["gene_name"]].shape

## yes: shape is the same

(16541, 7)

In [18]:
## thousands of unique values
df_no_ENSP_textmining["gene_id"].unique()

array(['18S_rRNA', '28S_rRNA', '45S_rRNA', ..., 'pRNA', 'snoU13',
       'snoU18'], shape=(3971,), dtype=object)

In [19]:
## look for other ID namespaces mentioned in paper

possible_gene_namespaces = ["ENSG", "ensembl", "uniprotkb", "uniprot", "hgnc"]
## remember function uses case=False (not sensitive)
check_if_contains(df_no_ENSP_textmining, "gene_id", possible_gene_namespaces)

## none: nothing prints

In [20]:
## double-check work on 1 value
df_textmining[df_textmining["gene_id"].str.contains("hgnc", case=False)]

,gene_id,gene_name,disease_id,disease_name,z_score,confidence_score,url


#### Knowledge

In [21]:
## quick look at knowledge data: short enough for manual review

## how much data doesn't have ENSP ID
df_knowledge[~ df_knowledge["gene_id"].str.startswith("ENSP")].shape

## vs entire dataset
df_knowledge[~ df_knowledge["gene_id"].str.startswith("ENSP")].shape[0] / df_knowledge.shape[0]

## unique values: can see that gene_id is a name (== gene_name), not an ID
df_knowledge[~ df_knowledge["gene_id"].str.startswith("ENSP")][["gene_id", "gene_name"]].drop_duplicates()

(41, 7)

0.005276705276705276

,gene_id,gene_name
0,ABHD11-AS1,ABHD11-AS1
7730,H19,H19
7734,KCNQ1OT1,KCNQ1OT1
7735,MIR17HG,MIR17HG
7736,MT-RNR1,MT-RNR1
7737,MT-TF,MT-TF
7738,MT-TH,MT-TH
7740,MT-TI,MT-TI
7742,MT-TK,MT-TK
7744,MT-TL1,MT-TL1


#### Experiments

In [25]:
# df_experiments[~ df_experiments["gene_id"].str.startswith("ENSP")].shape

### gene_name column

Didn't find IDs in this column that aren't in `gene_id`. 

Don't need this column for Translator output

In [22]:
delimiters
possible_gene_namespaces

[',', ';', ':']

['ENSG', 'ensembl', 'uniprotkb', 'uniprot', 'hgnc']

In [23]:
## look for delimiters

print("text-mining")
check_if_contains(df_textmining, "gene_name", delimiters)

print("knowledge")
check_if_contains(df_knowledge, "gene_name", delimiters)

## didn't print any stats, so not found

text-mining
knowledge


In [24]:
## look for other ID namespaces

print("text-mining")
check_if_contains(df_textmining, "gene_name", possible_gene_namespaces)

print("knowledge")
check_if_contains(df_knowledge, "gene_name", possible_gene_namespaces)

## didn't print any stats, so not found

text-mining
knowledge


In [25]:
## for textmining only, are there rows with ENSP in gene_name but not gene_id?
## not for knowledge: already reviewed manually

df_no_ENSP_textmining[df_no_ENSP_textmining["gene_name"].str.contains("ENSP", case=False)]
## no, empty df

,gene_id,gene_name,disease_id,disease_name,z_score,confidence_score,url


### disease_id

<div class="alert alert-block alert-info">

**A small portion of the data doesn't have DOID IDs!** Instead, they're AmyCo IDs (not usable). **Drop these rows from further parsing (can't NodeNorm)** 

In [26]:
## check for delimiters (between prefix and ID, between IDs)
## custom set since DOID: has :

print("text-mining")
check_if_contains(df_textmining, "disease_id", [",", ";", "_"])

print("knowledge")
check_if_contains(df_knowledge, "disease_id", [",", ";", "_"])

## didn't print any stats, so not found

text-mining
knowledge


#### Text-mining

In [27]:
## how much data doesn't have DOID ID
df_no_DOID_textmining = df_textmining[~ df_textmining["disease_id"].str.startswith("DOID:")].copy()
df_no_DOID_textmining.shape

## vs entire dataset
df_no_DOID_textmining.shape[0] / df_textmining.shape[0]

(42, 7)

0.00014537852066971038

In [28]:
## look at non-DOID data: unique values
df_no_DOID_textmining[["disease_id", "disease_name"]].drop_duplicates()

,disease_id,disease_name
51443,AmyCo:23,AH Amyloidosis
59481,AmyCo:35,Calcifying Epithelial Odontogenic Tumor
61486,AmyCo:73,Senile Seminal Vesicle Amyloidosis
79520,AmyCo:30,Wild-type transthyretin-related Amyloidosis
96710,AmyCo:27,Leukocyte chemotactic factor 2 Amyloidosis
127787,AmyCo:29,AA Amyloidosis
176742,AmyCo:37,Isolated Atrial Amyloidosis
187554,AmyCo:20,Hereditary lysozyme Amyloidosis
246435,AmyCo:24,AHL Amyloidosis
255280,AmyCo:8,Immunoglobulin Light-chain Amyloidosis


#### Knowledge

In [29]:
## how much data doesn't have DOID ID
df_no_DOID_knowledge = df_knowledge[~ df_knowledge["disease_id"].str.startswith("DOID:")].copy()
df_no_DOID_knowledge.shape

## vs entire dataset
df_no_DOID_knowledge.shape[0] / df_knowledge.shape[0]

(71, 7)

0.009137709137709138

In [30]:
## look at non-DOID data: unique values
df_no_DOID_knowledge[["disease_id", "disease_name"]].drop_duplicates()

,disease_id,disease_name
355,AmyCo:26,Apolipoprotein C-III associated Amyloidosis
511,AmyCo:16,Localized insulin-derived Amyloidosis
512,AmyCo:17,Apolipoprotein A-I associated Amyloidosis
514,AmyCo:63,Enfuvirtide-induced Amyloidosis
521,AmyCo:30,Wild-type transthyretin-related Amyloidosis
738,AmyCo:22,Apolipoprotein A-II associated Amyloidosis
739,AmyCo:23,AH Amyloidosis
740,AmyCo:24,AHL Amyloidosis
741,AmyCo:25,Apolipoprotein C-II associated Amyloidosis
743,AmyCo:27,Leukocyte chemotactic factor 2 Amyloidosis


#### Experiments

In [35]:
# df_experiments[~ df_experiments["disease_id"].str.startswith("DOID:")].shape

### Other columns

Quick checks on what other data looks like. 

Didn't review disease_name: already reviewed non-DOID rows, don't need in Translator output 

#### Text-mining

In [36]:
## minimum being 3 means filtering was probably done, to remove anything with low z-score
df_textmining["z_score"].describe()

## 1 = 2 standard deviations above mean, cutoff at 4
df_textmining["confidence_score"].describe()

df_textmining[df_textmining["confidence_score"] >= 3].shape

count    288901.000000
mean          3.900600
std           0.825751
min           3.000000
25%           3.279000
50%           3.664000
75%           4.281000
max          10.059000
Name: z_score, dtype: float64

count    288901.000000
mean          1.950139
std           0.412018
min           1.500000
25%           1.639000
50%           1.832000
75%           2.140000
max           4.000000
Name: confidence_score, dtype: float64

(8014, 7)

In [32]:
## url: check for delimiters (multiple urls)
## custom set since DOID: (:), some names have _

check_if_contains(df_textmining, "url", [",", ";"])

## didn't print any stats, so not found

In [33]:
## url: check for whitespace - none

df_textmining[df_textmining["url"].str.isspace()]

,gene_id,gene_name,disease_id,disease_name,z_score,confidence_score,url


#### Knowledge

In [37]:
df_knowledge["source_db"].value_counts()

source_db
MedlinePlus     3889
UniProtKB-KW    3624
AmyCo            257
Name: count, dtype: int64

In [38]:
df_knowledge["evidence_type"].value_counts()
## always the same value

evidence_type
CURATED    7770
Name: count, dtype: int64

In [39]:
df_knowledge["confidence_score"].value_counts()

confidence_score
5    3889
4    3830
3      35
2      16
Name: count, dtype: int64

#### Experiments

In [42]:
# df_experiments["confidence_score"].describe()

In [43]:
# check_if_contains(df_experiments, "source_score", ["MeanRankScore = "])

In [44]:
# df_experiments["MeanRankScore"] = df_experiments["source_score"].str.split(pat="MeanRankScore = ", expand=True)[1].astype("int")

In [45]:
# df_experiments["MeanRankScore"].describe()

## Remove rows

<div class="alert alert-block alert-info">

Removing:
* knowledge's UniProtKB-KW data: Doesn't seem accurate, Likely has licensing issues (OMIM). Analysis in Slack canvas
* rows that don't have ENSP IDs in gene_id column: can't NodeNorm them
* rows that don't have DOID IDs in disease_id column: can't NodeNorm them

    Not removing duplicates! Because planning to ingest file in chunks, so need other method (check set of already-done). 

In [40]:
## calculate stats before removing

nrows_textmining_original = df_textmining.shape[0]
nrows_knowledge_original = df_knowledge.shape[0]

In [41]:
## put into parser (don't do stats, print)

## remove knowledge's UniProtKB-KW data
df_knowledge = df_knowledge[df_knowledge["source_db"] != "UniProtKB-KW"]

## stats
nrows_knowledge_filtered = df_knowledge.shape[0]
print(f"knowledge: {nrows_knowledge_original - nrows_knowledge_filtered} rows removed")

knowledge: 3624 rows removed


In [42]:
## put into parser (don't do stats, print)

## remove rows that don't have ENSP IDs
df_textmining = df_textmining[df_textmining["gene_id"].str.startswith("ENSP")]
df_knowledge = df_knowledge[df_knowledge["gene_id"].str.startswith("ENSP")]


## stats
nrows_textmining_filtered = df_textmining.shape[0]
print(f"textmining: {nrows_textmining_original - nrows_textmining_filtered} rows removed")
## don't rewrite variable yet, so you can compare previous to now
print(f"knowledge: {nrows_knowledge_filtered - df_knowledge.shape[0]} rows removed")

## update this variable
nrows_knowledge_filtered = df_knowledge.shape[0]

textmining: 16541 rows removed
knowledge: 41 rows removed


In [43]:
## put into parser (don't do stats, print)

## remove rows that don't have DOID IDs
df_textmining = df_textmining[df_textmining["disease_id"].str.startswith("DOID:")]
df_knowledge = df_knowledge[df_knowledge["disease_id"].str.startswith("DOID:")]


## stats
## don't rewrite variable yet, so you can compare previous to now
print(f"textmining: {nrows_textmining_filtered - df_textmining.shape[0]} rows removed")
print(f"knowledge: {nrows_knowledge_filtered - df_knowledge.shape[0]} rows removed")

## update this variable
nrows_textmining_filtered = df_textmining.shape[0]
nrows_knowledge_filtered = df_knowledge.shape[0]

textmining: 42 rows removed
knowledge: 71 rows removed


In [44]:
## code to reviewing dataframes, make sure all removed properly

# df_textmining
# df_textmining[~ df_textmining["disease_id"].str.contains("DOID")]

# df_knowledge
# df_knowledge[~ df_knowledge["disease_id"].str.contains("DOID")]
# df_knowledge[df_knowledge["source_db"] == "UniProtKB-KW"]

## AmyCo data still in database (wasn't all under AmyCo disease IDs)
df_knowledge[df_knowledge["source_db"] == "AmyCo"]

,gene_id,gene_name,disease_id,disease_name,source_db,evidence_type,confidence_score
64,ENSP00000167586,KRT14,DOID:0050430,Multiple endocrine neoplasia type 2A,AmyCo,CURATED,2
65,ENSP00000167586,KRT14,DOID:0050639,Primary cutaneous amyloidosis,AmyCo,CURATED,4
67,ENSP00000167586,KRT14,DOID:2513,Basal cell carcinoma,AmyCo,CURATED,4
71,ENSP00000167586,KRT14,DOID:6498,Seborrheic keratosis,AmyCo,CURATED,4
72,ENSP00000167586,KRT14,DOID:7039,Borst-Jadassohn intraepidermal carcinoma,AmyCo,CURATED,3
...,...,...,...,...,...,...,...
7530,ENSP00000497221,ITM2B,DOID:0070029,ITM2B-related cerebral amyloid angiopathy 1,AmyCo,CURATED,4
7531,ENSP00000497221,ITM2B,DOID:0070030,ITM2B-related cerebral amyloid angiopathy 2,AmyCo,CURATED,4
7588,ENSP00000497910,B2M,DOID:0050747,DOID:0050747,AmyCo,CURATED,4
7717,ENSP00000500990,SNCA,DOID:12217,Lewy body dementia,AmyCo,CURATED,4


## Pre-NodeNorming

In [45]:
## put into parser (format)

## adding Translator/biolink prefixes to gene ID, for NodeNorming
## don't need to do DOID, already in right format

## loop for notebook only, to do both dataframes 
for df in [df_textmining, df_knowledge]:
    df["gene_id"] = "ENSEMBL:" + df["gene_id"]

In [46]:
## double-check how they look

df_knowledge

,gene_id,gene_name,disease_id,disease_name,source_db,evidence_type,confidence_score
2,ENSEMBL:ENSP00000003084,CFTR,DOID:0111862,Congenital bilateral absence of vas deferens,MedlinePlus,CURATED,5
3,ENSEMBL:ENSP00000003084,CFTR,DOID:1485,Cystic fibrosis,MedlinePlus,CURATED,5
4,ENSEMBL:ENSP00000005226,USH1C,DOID:0050439,Usher syndrome,MedlinePlus,CURATED,5
5,ENSEMBL:ENSP00000005226,USH1C,DOID:0050563,Nonsyndromic deafness,MedlinePlus,CURATED,5
11,ENSEMBL:ENSP00000013807,ERCC1,DOID:0050427,Xeroderma pigmentosum,MedlinePlus,CURATED,5
...,...,...,...,...,...,...,...
7723,ENSEMBL:ENSP00000500990,SNCA,DOID:4752,Multiple system atrophy,MedlinePlus,CURATED,5
7724,ENSEMBL:ENSP00000501092,TFAP2A,DOID:0050691,Branchiooculofacial syndrome,MedlinePlus,CURATED,5
7725,ENSEMBL:ENSP00000501092,TFAP2A,DOID:10629,Microphthalmia,MedlinePlus,CURATED,5
7726,ENSEMBL:ENSP00000501092,TFAP2A,DOID:12270,Coloboma,MedlinePlus,CURATED,5


Querying NodeNorm: send unique values (no duplicates!) in large batches -> generate mapping dict to use. 
<br>
__Not querying 1-by-1 or 1 row at a time: much slower__ and would involve sending duplicate IDs (unless saved dict is kept outside loop and checked) 

<div class="alert alert-block alert-danger">

Set the NodeNorm URL you want to use. 

In [47]:
## put into parser (format)

import requests

## from BioThings annotator code: for interoperability between diff Python versions
try:
    from itertools import batched  # new in Python 3.12
except ImportError:
    from itertools import islice

    def batched(iterable, n):
        # batched('ABCDEFG', 3) → ABC DEF G
        if n < 1:
            raise ValueError("n must be at least one")
        iterator = iter(iterable)
        while batch := tuple(islice(iterator, n)):
            yield batch

## dev instance
# nodenorm_url = "https://nodenormalization-sri.renci.org/get_normalized_nodes"
nodenorm_url = "https://nodenorm.ci.transltr.io/get_normalized_nodes"

### Genes

In [48]:
## put into parser (set unique on curie list)

## get set of unique CURIEs to put into NodeNorm
geneIDs_textmining = set(df_textmining["gene_id"].unique())
geneIDs_knowledge = set(df_knowledge["gene_id"].unique())
geneIDs_all = geneIDs_textmining | geneIDs_knowledge

len(geneIDs_textmining)
len(geneIDs_knowledge)
len(geneIDs_all)

16862

2567

16865

In [49]:
## put into parser (partial?)

gene_nodenorm_mapping = {}

## set up variables to catch potential mapping failures
stats_gene_mapping_failures = {
    "unexpected_error": {},
    "nodenorm_returned_none": [],
    "wrong_category": {},
    "no_label": []
}

Allowed types are Gene and Protein (depending on if gene/protein conflation worked for the ENSP ID or not)

In [50]:
## put into parser (don't need to iterate through batch, just use all in chunk you're on)

## larger batches are quicker
for batch in batched(geneIDs_all, 1000):
    ## returns tuples -> cast to list
    req_body = {
        "curies": list(batch),
        "conflate": True,
    }
    r = requests.post(nodenorm_url, json=req_body)
    response = r.json()
    
    ## not doing dict comprehension. allows easier review, logic writing
    for k,v in response.items():
        ## catch unexpected errors
        try:
            ## if NodeNorm didn't have info on this ID, v will be None
            if v is not None:
                ## don't keep mapping if category is not the expected one
                if v["type"][0] in ["biolink:Gene", "biolink:Protein"]:
                    ## also throw out mapping if no primary label found
                    if v["id"].get("label"):
                        temp = {
                            k: {"primary_id": v["id"]["identifier"],
                                "primary_label": v["id"]["label"],
                                ## saving category: currently required for KGX nodes file output
                                "category": v["type"][0]
                               }
                        }
                        gene_nodenorm_mapping.update(temp)
                    else:
                        stats_gene_mapping_failures["no_label"].append(k)
#                         print(f"{k}: NodeNorm didn't find primary label. Not keeping this mapping.")
                else:
                    stats_gene_mapping_failures["wrong_category"].update({k: v["type"][0]})
#                     print(f'{k}: NodeNorm found different category {v["type"][0]}. Not keeping this mapping.')
            else:
                stats_gene_mapping_failures["nodenorm_returned_none"].append(k)
#                 print(f"{k}: NodeNorm didn't recognize this ID")
        except:
            stats_gene_mapping_failures["unexpected_error"].update({k: v})
            print(f'Encountered an unexpected error.')
            print(f'NodeNorm response key: {k}')
            print(f'NodeNorm response value: {v}')

In [51]:
len(gene_nodenorm_mapping)

stats_gene_mapping_failures["unexpected_error"]

len(stats_gene_mapping_failures["nodenorm_returned_none"])
len(stats_gene_mapping_failures["wrong_category"])
len(stats_gene_mapping_failures["no_label"])

16482

{}

319

0

64

In [52]:
## code block for reviewing failures

# stats_gene_mapping_failures["nodenorm_returned_none"][270:280]
stats_gene_mapping_failures["no_label"][50:58]

['ENSEMBL:ENSP00000488912',
 'ENSEMBL:ENSP00000380440',
 'ENSEMBL:ENSP00000257177',
 'ENSEMBL:ENSP00000359579',
 'ENSEMBL:ENSP00000424176',
 'ENSEMBL:ENSP00000448665',
 'ENSEMBL:ENSP00000479693',
 'ENSEMBL:ENSP00000377616']

<div class="alert alert-block alert-info">

**2025-05-02 data:**

**Reviewed ENSEMBL ENSP mapping failures**

315 cases where NodeNorm returned None (didn't recognize/resolve ID). __I checked some (20)__ (0:10, 270:280). __Conclusion: DISEASES problem, using outdated IDs__

* 17: ID is deprecated. Language: "no longer in the database but it has been mapped to 1 deprecated identifier")
* 3: ID has been replaced? Language: "no longer in the database.It has been mapped to 1 current identifier". (ENSP00000074304, ENSP00000480798, ENSP00000482552)

<br>
    
58 cases where NodeNorm didn't have a primary label. __I checked some (18)__ (0:10, 50:58). __Conclusion: NodeNorm issue sometimes, other times a complex biology/data situation__

All were legit IDs, but I guess NodeNorm isn't mapping them to their genes. Some thoughts:
* noticed many genes were overlapping locus/readthrough gene
* But some of these genes look "fairly regular" and the ENSP ID is the "Ensembl Canonical". So it's strange that NodeNorm isn't mapping them to their genes -> messaged NodeNorm
  * ENSP00000419081 for CALML4 ENSG00000129007 
  * ENSP00000211076 for TPSD1 ENSG00000095917 (not overlapping!)
  * ENSP00000361746 for EPPIN ENSG00000101448
  * ENSP00000215727 for SERPIND1 ENSG00000099937 (not overlapping!)
  * ENSP00000348089 for ERCC6 ENSG00000225830 (not overlapping!)
  * ENSP00000361232 for DDX31 ENSG00000125485 (not overlapping!)

In [53]:
## calculate stats: number of rows affected by each type of mapping failure
stats_gene_mapping_failures.update({
    "nrows_textmining_none": df_textmining[df_textmining["gene_id"].isin(stats_gene_mapping_failures["nodenorm_returned_none"])].shape[0],
    "nrows_knowledge_none": df_knowledge[df_knowledge["gene_id"].isin(stats_gene_mapping_failures["nodenorm_returned_none"])].shape[0],

    "n_rows_textmining_wrongcategory": df_textmining[df_textmining["gene_id"].isin(stats_gene_mapping_failures["wrong_category"].keys())].shape[0],
    "n_rows_knowledge_wrongcategory": df_knowledge[df_knowledge["gene_id"].isin(stats_gene_mapping_failures["wrong_category"].keys())].shape[0],

    
    "n_rows_textmining_nolabel": df_textmining[df_textmining["gene_id"].isin(stats_gene_mapping_failures["no_label"])].shape[0],
    "n_rows_knowledge_nolabel": df_knowledge[df_knowledge["gene_id"].isin(stats_gene_mapping_failures["no_label"])].shape[0],
})

In [54]:
stats_gene_mapping_failures["nrows_textmining_none"]
stats_gene_mapping_failures["nrows_knowledge_none"]
stats_gene_mapping_failures["n_rows_textmining_wrongcategory"]
stats_gene_mapping_failures["n_rows_knowledge_wrongcategory"]
stats_gene_mapping_failures["n_rows_textmining_nolabel"]
stats_gene_mapping_failures["n_rows_knowledge_nolabel"]

4238

54

0

0

1450

11

### Diseases

In [55]:

## get set of unique CURIEs to put into NodeNorm
diseaseIDs_textmining = set(df_textmining["disease_id"].unique())
diseaseIDs_knowledge = set(df_knowledge["disease_id"].unique())
diseaseIDs_all = diseaseIDs_textmining | diseaseIDs_knowledge

len(diseaseIDs_textmining)
len(diseaseIDs_knowledge)
len(diseaseIDs_all)

5561

1037

5676

In [56]:

disease_nodenorm_mapping = {}

## set up variables to catch mapping failures
stats_disease_mapping_failures = {
    "unexpected_error": {},
    "nodenorm_returned_none": [],
    "wrong_category": {},
    "no_label": []
    
}

In [57]:


## larger batches are quicker
for batch in batched(diseaseIDs_all, 1000):
    ## returns tuples -> cast to list
    req_body = {
        "curies": list(batch),
        "conflate": True,
    }
    r = requests.post(nodenorm_url, json=req_body)
    response = r.json()
    
    ## not doing dict comprehension. allows easier review, logic writing
    for k,v in response.items():
        ## catch unexpected errors
        try:
            ## if NodeNorm didn't have info on this ID, v will be None
            if v is not None:
                ## don't keep mapping if category is not the expected one
                if v["type"][0] == "biolink:Disease":
                    ## also throw out mapping if no primary label found
                    if v["id"].get("label"):
                        temp = {
                            k: {"primary_id": v["id"]["identifier"],
                                "primary_label": v["id"]["label"]
                               }
                        }
                        disease_nodenorm_mapping.update(temp)
                    else:
                        stats_disease_mapping_failures["no_label"].append(k)
#                         print(f"{k}: NodeNorm didn't find primary label. Not keeping this mapping.")
                else:
                    stats_disease_mapping_failures["wrong_category"].update({k: v["type"][0]})
#                     print(f'{k}: NodeNorm found different category {v["type"][0]}. Not keeping this mapping.')
            else:
                stats_disease_mapping_failures["nodenorm_returned_none"].append(k)
#                 print(f"{k}: NodeNorm didn't recognize this ID")
        except:
            stats_disease_mapping_failures["unexpected_error"].update({k: v})
            print(f'Encountered an unexpected error.')
            print(f'NodeNorm response key: {k}')
            print(f'NodeNorm response value: {v}')

In [58]:
len(disease_nodenorm_mapping)

stats_disease_mapping_failures["unexpected_error"]

len(stats_disease_mapping_failures["nodenorm_returned_none"])
len(stats_disease_mapping_failures["wrong_category"])
len(stats_disease_mapping_failures["no_label"])

5646

{}

30

0

0

In [59]:
## code used to review mapping failures 

stats_disease_mapping_failures["nodenorm_returned_none"][0:10]

['DOID:0060320',
 'DOID:1086',
 'DOID:2848',
 'DOID:11282',
 'DOID:9341',
 'DOID:0110578',
 'DOID:11772',
 'DOID:0050747',
 'DOID:10986',
 'DOID:0081164']

<div class="alert alert-block alert-info">

**2025-05-02 data:**

**Reviewed Disease DOID mapping failures**

30 cases where NodeNorm returned None (didn't recognize/resolve ID). __I checked some (first 10)__:

All were legit IDs: don't understand why NodeNorm returned none -> messaged NodeNorm

In [60]:
## calculate stats: number of rows affected by each type of mapping failure
stats_disease_mapping_failures.update({
    "nrows_textmining_none": df_textmining[df_textmining["disease_id"].isin(stats_disease_mapping_failures["nodenorm_returned_none"])].shape[0],
    "nrows_knowledge_none": df_knowledge[df_knowledge["disease_id"].isin(stats_disease_mapping_failures["nodenorm_returned_none"])].shape[0],

    "n_rows_textmining_wrongcategory": df_textmining[df_textmining["disease_id"].isin(stats_disease_mapping_failures["wrong_category"].keys())].shape[0],
    "n_rows_knowledge_wrongcategory": df_knowledge[df_knowledge["disease_id"].isin(stats_disease_mapping_failures["wrong_category"].keys())].shape[0],

    
    "n_rows_textmining_nolabel": df_textmining[df_textmining["disease_id"].isin(stats_disease_mapping_failures["no_label"])].shape[0],
    "n_rows_knowledge_nolabel": df_knowledge[df_knowledge["disease_id"].isin(stats_disease_mapping_failures["no_label"])].shape[0],
})

In [61]:
stats_disease_mapping_failures["nrows_textmining_none"]
stats_disease_mapping_failures["nrows_knowledge_none"]
stats_disease_mapping_failures["n_rows_textmining_wrongcategory"]
stats_disease_mapping_failures["n_rows_knowledge_wrongcategory"]
stats_disease_mapping_failures["n_rows_textmining_nolabel"]
stats_disease_mapping_failures["n_rows_knowledge_nolabel"]

628

10

0

0

0

0

### Adding NodeNorm data, removing rows

<div class="alert alert-block alert-info">

Removing rows that lack NodeNorm data, due to small amount of failures during NodeNorm process: 
* NodeNorm returned none
* NodeNorm didn't have primary label (genes only) 

In [62]:
## put into parser (format): 

## text-mining
df_textmining["gene_nodenorm_id"] = [gene_nodenorm_mapping[i]["primary_id"] 
                                     if gene_nodenorm_mapping.get(i) 
                                     else pd.NA
                                     for i in df_textmining["gene_id"]]

df_textmining["gene_nodenorm_label"] = [gene_nodenorm_mapping[i]["primary_label"] 
                                        if gene_nodenorm_mapping.get(i) 
                                        else pd.NA
                                        for i in df_textmining["gene_id"]]

df_textmining["gene_nodenorm_category"] = [gene_nodenorm_mapping[i]["category"] 
                                           if gene_nodenorm_mapping.get(i) 
                                           else pd.NA
                                           for i in df_textmining["gene_id"]]

df_textmining["disease_nodenorm_id"] = [disease_nodenorm_mapping[i]["primary_id"] 
                                        if disease_nodenorm_mapping.get(i) 
                                        else pd.NA
                                        for i in df_textmining["disease_id"]]

df_textmining["disease_nodenorm_label"] = [disease_nodenorm_mapping[i]["primary_label"] 
                                           if disease_nodenorm_mapping.get(i) 
                                           else pd.NA
                                           for i in df_textmining["disease_id"]]

In [63]:

## knowledge
df_knowledge["gene_nodenorm_id"] = [gene_nodenorm_mapping[i]["primary_id"] 
                                    if gene_nodenorm_mapping.get(i) 
                                    else pd.NA
                                    for i in df_knowledge["gene_id"]]

df_knowledge["gene_nodenorm_label"] = [gene_nodenorm_mapping[i]["primary_label"] 
                                       if gene_nodenorm_mapping.get(i) 
                                       else pd.NA
                                       for i in df_knowledge["gene_id"]]

df_knowledge["gene_nodenorm_category"] = [gene_nodenorm_mapping[i]["category"] 
                                          if gene_nodenorm_mapping.get(i) 
                                          else pd.NA
                                          for i in df_knowledge["gene_id"]]

df_knowledge["disease_nodenorm_id"] = [disease_nodenorm_mapping[i]["primary_id"] 
                                       if disease_nodenorm_mapping.get(i) 
                                       else pd.NA
                                       for i in df_knowledge["disease_id"]]

df_knowledge["disease_nodenorm_label"] = [disease_nodenorm_mapping[i]["primary_label"] 
                                          if disease_nodenorm_mapping.get(i) 
                                          else pd.NA
                                          for i in df_knowledge["disease_id"]]

In [70]:
## code chunk to review df

# df_textmining
# df_knowledge

In [64]:
## put into parser: 

## doing subset just in case (original data didn't have any NAs that would be
##   dropped on accident, but still) 

df_textmining.dropna(subset=["gene_nodenorm_id", "gene_nodenorm_label", 
                            "disease_nodenorm_id", "disease_nodenorm_label"],
                    ignore_index=True, inplace=True)

df_knowledge.dropna(subset=["gene_nodenorm_id", "gene_nodenorm_label", 
                            "disease_nodenorm_id", "disease_nodenorm_label"],
                    ignore_index=True, inplace=True)

In [65]:
## didn't do stats for before/after

df_textmining.shape
df_knowledge.shape

(266014, 12)

(3960, 12)

### Exploring "duplicates" from NodeNorming

In [66]:
## number of rows affected

df_knowledge_noTotalDups = df_knowledge.drop_duplicates()

df_knowledge_nodenorm_dups = df_knowledge_noTotalDups[
    df_knowledge_noTotalDups.duplicated(
        subset=["gene_nodenorm_id", "disease_nodenorm_id", "source_db"], 
        keep=False)].copy()

df_knowledge_nodenorm_dups.sort_values(by=["disease_nodenorm_id", "gene_nodenorm_id"],
                                       inplace=True)

df_knowledge_nodenorm_dups.shape

df_knowledge_nodenorm_dups

(10, 12)

,gene_id,gene_name,disease_id,disease_name,source_db,evidence_type,confidence_score,gene_nodenorm_id,gene_nodenorm_label,gene_nodenorm_category,disease_nodenorm_id,disease_nodenorm_label
2612,ENSEMBL:ENSP00000370083,SMN1,DOID:12377,Spinal muscular atrophy,MedlinePlus,CURATED,5,UniProtKB:Q16637-1,survival motor neuron protein isoform 1 (human),biolink:Protein,MONDO:0001516,spinal muscular atrophy
2615,ENSEMBL:ENSP00000370119,SMN2,DOID:12377,Spinal muscular atrophy,MedlinePlus,CURATED,5,UniProtKB:Q16637-1,survival motor neuron protein isoform 1 (human),biolink:Protein,MONDO:0001516,spinal muscular atrophy
119,ENSEMBL:ENSP00000221700,CYP4F2,DOID:0080665,Warfarin resistance,MedlinePlus,CURATED,5,UniProtKB:P78329-1,cytochrome P450 4F2 isoform h1 (human),biolink:Protein,MONDO:0007390,coumarin resistance
120,ENSEMBL:ENSP00000221700,CYP4F2,DOID:0080666,Warfarin sensitivity,MedlinePlus,CURATED,5,UniProtKB:P78329-1,cytochrome P450 4F2 isoform h1 (human),biolink:Protein,MONDO:0007390,coumarin resistance
2769,ENSEMBL:ENSP00000378426,VKORC1,DOID:0080665,Warfarin resistance,MedlinePlus,CURATED,5,UniProtKB:Q9BQB6-1,vitamin K epoxide reductase complex subunit 1 ...,biolink:Protein,MONDO:0007390,coumarin resistance
2770,ENSEMBL:ENSP00000378426,VKORC1,DOID:0080666,Warfarin sensitivity,MedlinePlus,CURATED,5,UniProtKB:Q9BQB6-1,vitamin K epoxide reductase complex subunit 1 ...,biolink:Protein,MONDO:0007390,coumarin resistance
344,ENSEMBL:ENSP00000251595,HBA2,DOID:1099,Alpha thalassemia,MedlinePlus,CURATED,5,UniProtKB:P69905,HBA_HUMAN Hemoglobin subunit alpha (sprot),biolink:Protein,MONDO:0011399,alpha thalassemia spectrum
1422,ENSEMBL:ENSP00000322421,HBA1,DOID:1099,Alpha thalassemia,MedlinePlus,CURATED,5,UniProtKB:P69905,HBA_HUMAN Hemoglobin subunit alpha (sprot),biolink:Protein,MONDO:0011399,alpha thalassemia spectrum
2281,ENSEMBL:ENSP00000361021,PTEN,DOID:0050657,Bannayan-Riley-Ruvalcaba syndrome,MedlinePlus,CURATED,5,UniProtKB:P60484-1,"phosphatidylinositol 3,4,5-trisphosphate 3-pho...",biolink:Protein,MONDO:0016063,Cowden disease
2289,ENSEMBL:ENSP00000361021,PTEN,DOID:6457,Cowden syndrome,MedlinePlus,CURATED,5,UniProtKB:P60484-1,"phosphatidylinositol 3,4,5-trisphosphate 3-pho...",biolink:Protein,MONDO:0016063,Cowden disease


In [67]:
## knowledge disease duplicates

NodeNormDups_kw_diseases = df_knowledge_nodenorm_dups[["disease_id", "disease_name", 
                            "disease_nodenorm_id", "disease_nodenorm_label"]].copy()

NodeNormDups_kw_diseases.drop_duplicates(inplace=True)

diseases_dups_kw = NodeNormDups_kw_diseases.groupby(
    "disease_nodenorm_id").filter(lambda x: len(x)>1).copy()

diseases_dups_kw

,disease_id,disease_name,disease_nodenorm_id,disease_nodenorm_label
119,DOID:0080665,Warfarin resistance,MONDO:0007390,coumarin resistance
120,DOID:0080666,Warfarin sensitivity,MONDO:0007390,coumarin resistance
2281,DOID:0050657,Bannayan-Riley-Ruvalcaba syndrome,MONDO:0016063,Cowden disease
2289,DOID:6457,Cowden syndrome,MONDO:0016063,Cowden disease


In [68]:
## knowledge gene duplicates

NodeNormDups_kw_genes = df_knowledge_nodenorm_dups[["gene_id", "gene_name", 
                            "gene_nodenorm_id", "gene_nodenorm_label"]].copy()

NodeNormDups_kw_genes.drop_duplicates(inplace=True)

genes_dups_kw = NodeNormDups_kw_genes.groupby(
    "gene_nodenorm_id").filter(lambda x: len(x)>1).copy()

genes_dups_kw

,gene_id,gene_name,gene_nodenorm_id,gene_nodenorm_label
2612,ENSEMBL:ENSP00000370083,SMN1,UniProtKB:Q16637-1,survival motor neuron protein isoform 1 (human)
2615,ENSEMBL:ENSP00000370119,SMN2,UniProtKB:Q16637-1,survival motor neuron protein isoform 1 (human)
344,ENSEMBL:ENSP00000251595,HBA2,UniProtKB:P69905,HBA_HUMAN Hemoglobin subunit alpha (sprot)
1422,ENSEMBL:ENSP00000322421,HBA1,UniProtKB:P69905,HBA_HUMAN Hemoglobin subunit alpha (sprot)


In [69]:
df_textmining_nodenorm_dups = df_textmining[
    df_textmining.duplicated(
        subset=["gene_nodenorm_id", "disease_nodenorm_id"], 
        keep=False)].copy()

df_textmining_nodenorm_dups.sort_values(by=["disease_nodenorm_id", "gene_nodenorm_id"],
                                        inplace=True)

df_textmining_nodenorm_dups.shape

df_textmining_nodenorm_dups

(1112, 12)

,gene_id,gene_name,disease_id,disease_name,z_score,confidence_score,url,gene_nodenorm_id,gene_nodenorm_label,gene_nodenorm_category,disease_nodenorm_id,disease_nodenorm_label
181834,ENSEMBL:ENSP00000370083,SMN1,DOID:0060159,Organic acidemia,3.414,1.707,https://diseases.jensenlab.org/Entity?document...,UniProtKB:Q16637-1,survival motor neuron protein isoform 1 (human),biolink:Protein,MONDO:0000688,inborn organic aciduria
181951,ENSEMBL:ENSP00000370119,SMN2,DOID:0060159,Organic acidemia,3.448,1.724,https://diseases.jensenlab.org/Entity?document...,UniProtKB:Q16637-1,survival motor neuron protein isoform 1 (human),biolink:Protein,MONDO:0000688,inborn organic aciduria
188722,ENSEMBL:ENSP00000376048,MAG,DOID:0060499,Autoimmune neuropathy,4.996,2.498,https://diseases.jensenlab.org/Entity?document...,UniProtKB:P20916-1,myelin-associated glycoprotein isoform 1 (human),biolink:Protein,MONDO:0000774,autoimmune neuropathy
188733,ENSEMBL:ENSP00000376048,MAG,DOID:0040087,Autoimmune peripheral neuropathy,4.178,2.089,https://diseases.jensenlab.org/Entity?document...,UniProtKB:P20916-1,myelin-associated glycoprotein isoform 1 (human),biolink:Protein,MONDO:0000774,autoimmune neuropathy
116656,ENSEMBL:ENSP00000334330,DEFB105A,DOID:1002,Endometritis,3.510,1.755,https://diseases.jensenlab.org/Entity?document...,UniProtKB:Q8NG35,D105A_HUMAN Beta-defensin 105 (sprot),biolink:Protein,MONDO:0000918,endometritis
...,...,...,...,...,...,...,...,...,...,...,...,...
145083,ENSEMBL:ENSP00000355780,H3-3A,DOID:0081302,Diffuse leptomeningeal glioneuronal tumor,4.208,2.104,https://diseases.jensenlab.org/Entity?document...,UniProtKB:P84243,H33_HUMAN Histone H3.3 (sprot),biolink:Protein,MONDO:0858956,diffuse leptomeningeal glioneuronal tumor
116657,ENSEMBL:ENSP00000334330,DEFB105A,DOID:0070468,Yoon-Bellen neurodevelopmental syndrome,3.238,1.619,https://diseases.jensenlab.org/Entity?document...,UniProtKB:Q8NG35,D105A_HUMAN Beta-defensin 105 (sprot),biolink:Protein,MONDO:0859221,Yoon-Bellen neurodevelopmental syndrome
117572,ENSEMBL:ENSP00000335281,DEFB105B,DOID:0070468,Yoon-Bellen neurodevelopmental syndrome,3.590,1.795,https://diseases.jensenlab.org/Entity?document...,UniProtKB:Q8NG35,D105A_HUMAN Beta-defensin 105 (sprot),biolink:Protein,MONDO:0859221,Yoon-Bellen neurodevelopmental syndrome
103533,ENSEMBL:ENSP00000320813,DEFB104A,DOID:0070468,Yoon-Bellen neurodevelopmental syndrome,4.116,2.058,https://diseases.jensenlab.org/Entity?document...,UniProtKB:Q8WTQ1,D104A_HUMAN Beta-defensin 104 (sprot),biolink:Protein,MONDO:0859221,Yoon-Bellen neurodevelopmental syndrome


In [70]:
df_textmining_nodenorm_dups[df_textmining_nodenorm_dups["disease_nodenorm_id"] == "MONDO:0005689"]

list(df_textmining_nodenorm_dups[df_textmining_nodenorm_dups["disease_nodenorm_id"] == "MONDO:0005689"]["url"])

,gene_id,gene_name,disease_id,disease_name,z_score,confidence_score,url,gene_nodenorm_id,gene_nodenorm_label,gene_nodenorm_category,disease_nodenorm_id,disease_nodenorm_label
230110,ENSEMBL:ENSP00000427603,GABRA2,DOID:1849,Cannabis dependence,5.000,2.500,https://diseases.jensenlab.org/Entity?document...,NCBIGene:2555,GABRA2,biolink:Gene,MONDO:0005689,cannabis abuse
230123,ENSEMBL:ENSP00000427603,GABRA2,DOID:9505,Cannabis abuse,3.537,1.768,https://diseases.jensenlab.org/Entity?document...,NCBIGene:2555,GABRA2,biolink:Gene,MONDO:0005689,cannabis abuse
216628,ENSEMBL:ENSP00000402020,C17orf58,DOID:1849,Cannabis dependence,4.145,2.073,https://diseases.jensenlab.org/Entity?document...,NCBIGene:284018,C17orf58,biolink:Gene,MONDO:0005689,cannabis abuse
216629,ENSEMBL:ENSP00000402020,C17orf58,DOID:9505,Cannabis abuse,3.363,1.681,https://diseases.jensenlab.org/Entity?document...,NCBIGene:284018,C17orf58,biolink:Gene,MONDO:0005689,cannabis abuse
202599,ENSEMBL:ENSP00000384193,CADM2,DOID:1849,Cannabis dependence,4.422,2.211,https://diseases.jensenlab.org/Entity?document...,UniProtKB:Q8N3J6-3,cell adhesion molecule 2 isoform h3 (human),biolink:Protein,MONDO:0005689,cannabis abuse
202614,ENSEMBL:ENSP00000384193,CADM2,DOID:9505,Cannabis abuse,3.016,1.508,https://diseases.jensenlab.org/Entity?document...,UniProtKB:Q8N3J6-3,cell adhesion molecule 2 isoform h3 (human),biolink:Protein,MONDO:0005689,cannabis abuse


['https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000427603&type2=-26&id2=DOID:1849',
 'https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000427603&type2=-26&id2=DOID:9505',
 'https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000402020&type2=-26&id2=DOID:1849',
 'https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000402020&type2=-26&id2=DOID:9505',
 'https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000384193&type2=-26&id2=DOID:1849',
 'https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000384193&type2=-26&id2=DOID:9505']

In [71]:
## knowledge disease duplicates

NodeNormDups_tm_diseases = df_textmining_nodenorm_dups[["disease_id", "disease_name", 
                            "disease_nodenorm_id", "disease_nodenorm_label"]].copy()

NodeNormDups_tm_diseases.drop_duplicates(inplace=True)

diseases_dups_tm = NodeNormDups_tm_diseases.groupby(
    "disease_nodenorm_id").filter(lambda x: len(x)>1).copy()

diseases_dups_tm["disease_nodenorm_id"].nunique()
diseases_dups_tm

13

,disease_id,disease_name,disease_nodenorm_id,disease_nodenorm_label
188722,DOID:0060499,Autoimmune neuropathy,MONDO:0000774,autoimmune neuropathy
188733,DOID:0040087,Autoimmune peripheral neuropathy,MONDO:0000774,autoimmune neuropathy
120605,DOID:0060025,Immunoglobulin alpha deficiency,MONDO:0001341,IgA Deficiency
120636,DOID:11701,Selective IgA deficiency disease,MONDO:0001341,IgA Deficiency
86498,DOID:824,Periodontitis,MONDO:0005076,periodontitis
162278,DOID:9893,Periodontosis,MONDO:0005076,periodontitis
80373,DOID:1575,Rheumatic disease,MONDO:0005554,Rheumatism
80381,DOID:854,Collagen disease,MONDO:0005554,Rheumatism
230110,DOID:1849,Cannabis dependence,MONDO:0005689,cannabis abuse
230123,DOID:9505,Cannabis abuse,MONDO:0005689,cannabis abuse


In [72]:
## knowledge gene duplicates

NodeNormDups_tm_genes = df_textmining_nodenorm_dups[["gene_id", "gene_name", 
                            "gene_nodenorm_id", "gene_nodenorm_label"]].copy()

NodeNormDups_tm_genes.drop_duplicates(inplace=True)

genes_dups_tm = NodeNormDups_tm_genes.groupby(
    "gene_nodenorm_id").filter(lambda x: len(x)>1).copy()

genes_dups_tm["gene_nodenorm_id"].nunique()
genes_dups_tm

36

,gene_id,gene_name,gene_nodenorm_id,gene_nodenorm_label
181834,ENSEMBL:ENSP00000370083,SMN1,UniProtKB:Q16637-1,survival motor neuron protein isoform 1 (human)
181951,ENSEMBL:ENSP00000370119,SMN2,UniProtKB:Q16637-1,survival motor neuron protein isoform 1 (human)
116656,ENSEMBL:ENSP00000334330,DEFB105A,UniProtKB:Q8NG35,D105A_HUMAN Beta-defensin 105 (sprot)
117571,ENSEMBL:ENSP00000335281,DEFB105B,UniProtKB:Q8NG35,D105A_HUMAN Beta-defensin 105 (sprot)
228343,ENSEMBL:ENSP00000421861,MAGED4B,UniProtKB:Q96JG8-4,melanoma-associated antigen D4 isoform h4 (human)
...,...,...,...,...
111878,ENSEMBL:ENSP00000329982,OR4F16,UniProtKB:Q6IEY1,OR4F3_HUMAN Olfactory receptor 4F3/4F16/4F29 (...
212072,ENSEMBL:ENSP00000394282,OR4F3,UniProtKB:Q6IEY1,OR4F3_HUMAN Olfactory receptor 4F3/4F16/4F29 (...
220046,ENSEMBL:ENSP00000409316,OR4F29,UniProtKB:Q6IEY1,OR4F3_HUMAN Olfactory receptor 4F3/4F16/4F29 (...
1164,ENSEMBL:ENSP00000016946,RGPD5,UniProtKB:Q99666-1,RANBP2-like and GRIP domain-containing protein...


In [73]:
genes_dups_tm[60:90]

,gene_id,gene_name,gene_nodenorm_id,gene_nodenorm_label
249647,ENSEMBL:ENSP00000479461,H4C4,UniProtKB:P62805,H4_HUMAN Histone H4 (sprot)
249819,ENSEMBL:ENSP00000479794,H4C12,UniProtKB:P62805,H4_HUMAN Histone H4 (sprot)
250997,ENSEMBL:ENSP00000480960,H4C13,UniProtKB:P62805,H4_HUMAN Histone H4 (sprot)
251378,ENSEMBL:ENSP00000481486,H4C9,UniProtKB:P62805,H4_HUMAN Histone H4 (sprot)
254101,ENSEMBL:ENSP00000484789,H4C5,UniProtKB:P62805,H4_HUMAN Histone H4 (sprot)
104584,ENSEMBL:ENSP00000321744,H2BC4,UniProtKB:P62807,H2B1C_HUMAN Histone H2B type 1-C/E/F/G/I (sprot)
132337,ENSEMBL:ENSP00000348924,H2BC7,UniProtKB:P62807,H2B1C_HUMAN Histone H2B type 1-C/E/F/G/I (sprot)
174054,ENSEMBL:ENSP00000366962,H2BC10,UniProtKB:P62807,H2B1C_HUMAN Histone H2B type 1-C/E/F/G/I (sprot)
238756,ENSEMBL:ENSP00000445633,H2BC8,UniProtKB:P62807,H2B1C_HUMAN Histone H2B type 1-C/E/F/G/I (sprot)
254921,ENSEMBL:ENSP00000489317,H2BC6,UniProtKB:P62807,H2B1C_HUMAN Histone H2B type 1-C/E/F/G/I (sprot)


In [74]:
diseases_dups_final = pd.concat(
    [diseases_dups_kw, diseases_dups_tm], 
    axis=0, 
    ignore_index=True).drop_duplicates()

diseases_dups_final.columns = ["input_id", "input_name", 
                               "nodenorm_id", "nodenorm_name"]

diseases_dups_final.shape
diseases_dups_final["nodenorm_id"].nunique()
diseases_dups_final



## file path
output_base_path = pathlib.Path.home().joinpath("Desktop", "DISEASES_files")
diseases_output_path = output_base_path.joinpath("disease_nodenorm_dups.txt")



# diseases_dups_final.to_csv(diseases_output_path, sep="\t")

(29, 4)

14

,input_id,input_name,nodenorm_id,nodenorm_name
0,DOID:0080665,Warfarin resistance,MONDO:0007390,coumarin resistance
1,DOID:0080666,Warfarin sensitivity,MONDO:0007390,coumarin resistance
2,DOID:0050657,Bannayan-Riley-Ruvalcaba syndrome,MONDO:0016063,Cowden disease
3,DOID:6457,Cowden syndrome,MONDO:0016063,Cowden disease
4,DOID:0060499,Autoimmune neuropathy,MONDO:0000774,autoimmune neuropathy
5,DOID:0040087,Autoimmune peripheral neuropathy,MONDO:0000774,autoimmune neuropathy
6,DOID:0060025,Immunoglobulin alpha deficiency,MONDO:0001341,IgA Deficiency
7,DOID:11701,Selective IgA deficiency disease,MONDO:0001341,IgA Deficiency
8,DOID:824,Periodontitis,MONDO:0005076,periodontitis
9,DOID:9893,Periodontosis,MONDO:0005076,periodontitis


In [75]:
genes_dups_final = pd.concat(
    [genes_dups_kw, genes_dups_tm], 
    axis=0, 
    ignore_index=True).drop_duplicates()

genes_dups_final.columns = ["input_id", "input_name", 
                               "nodenorm_id", "nodenorm_name"]

genes_dups_final.shape
genes_dups_final["nodenorm_id"].nunique()
genes_dups_final

## file path
output_base_path = pathlib.Path.home().joinpath("Desktop", "DISEASES_files")
genes_output_path = output_base_path.joinpath("gene_nodenorm_dups.txt")



# genes_dups_final.to_csv(genes_output_path, sep="\t")

(106, 4)

36

,input_id,input_name,nodenorm_id,nodenorm_name
0,ENSEMBL:ENSP00000370083,SMN1,UniProtKB:Q16637-1,survival motor neuron protein isoform 1 (human)
1,ENSEMBL:ENSP00000370119,SMN2,UniProtKB:Q16637-1,survival motor neuron protein isoform 1 (human)
2,ENSEMBL:ENSP00000251595,HBA2,UniProtKB:P69905,HBA_HUMAN Hemoglobin subunit alpha (sprot)
3,ENSEMBL:ENSP00000322421,HBA1,UniProtKB:P69905,HBA_HUMAN Hemoglobin subunit alpha (sprot)
6,ENSEMBL:ENSP00000334330,DEFB105A,UniProtKB:Q8NG35,D105A_HUMAN Beta-defensin 105 (sprot)
...,...,...,...,...
105,ENSEMBL:ENSP00000329982,OR4F16,UniProtKB:Q6IEY1,OR4F3_HUMAN Olfactory receptor 4F3/4F16/4F29 (...
106,ENSEMBL:ENSP00000394282,OR4F3,UniProtKB:Q6IEY1,OR4F3_HUMAN Olfactory receptor 4F3/4F16/4F29 (...
107,ENSEMBL:ENSP00000409316,OR4F29,UniProtKB:Q6IEY1,OR4F3_HUMAN Olfactory receptor 4F3/4F16/4F29 (...
108,ENSEMBL:ENSP00000016946,RGPD5,UniProtKB:Q99666-1,RANBP2-like and GRIP domain-containing protein...


## Generating documents

In [76]:
## put in parser!!
## want jsonlines format

import jsonlines

### Rows not included

<div class="alert alert-block alert-info">

* knowledge's UniProtKB-KW data
* No ENSP in gene_id columns (seemed to be names, couldn't NodeNorm)
* No DOID in disease_id columns (can't NodeNorm AmyCo)
* NodeNorm mapping failures for gene or disease IDs
* duplicates: will check when generating docs, not create if already did

### Columns not included

<div class="alert alert-block alert-info">

- gene_name
- disease_name
- evidence_type: same for all rows, not needed

In [77]:
df_textmining.columns

df_knowledge.columns

Index(['gene_id', 'gene_name', 'disease_id', 'disease_name', 'z_score',
       'confidence_score', 'url', 'gene_nodenorm_id', 'gene_nodenorm_label',
       'gene_nodenorm_category', 'disease_nodenorm_id',
       'disease_nodenorm_label'],
      dtype='object')

Index(['gene_id', 'gene_name', 'disease_id', 'disease_name', 'source_db',
       'evidence_type', 'confidence_score', 'gene_nodenorm_id',
       'gene_nodenorm_label', 'gene_nodenorm_category', 'disease_nodenorm_id',
       'disease_nodenorm_label'],
      dtype='object')

### File: List of TRAPI edges 

In [ ]:
## code chunk for testing parts of inner code

# with jsonlines.open('DISEASES_trapi_edges.jsonl', mode='w', compact=True) as trapi_writer:
#     knowledge_tally = set()
    
#     ## using itertuples because it's faster, preserves datatypes
#     for row in df_knowledge.itertuples(index=False):

#             break

In [78]:
## put into parser (format): 
## separate functions for textmined vs not, trapi edges vs kgx??
## will want the tally to live outside the reading data -> writing loop, so it can
##   keep track of what edges were already done

## if needed to create two diff output file formats at same time, could initiate
##   to separate writers and use both at once 


with jsonlines.open('DISEASES_trapi_edges.jsonl', mode='w', compact=True) as trapi_writer:
    
## text-mined data: 
    textmining_tally = set()
    
    ## using itertuples because it's faster, preserves datatypes
    for row in df_textmining.itertuples(index=False):
        ## construct row abbreviation
        temp = row.gene_id + "_" + row.disease_id
        
        if temp not in textmining_tally:
            textmining_tally.add(temp)

            document = {
                "subject": row.gene_nodenorm_id,
                "predicate": "biolink:occurs_together_in_literature_with",
                "object": row.disease_nodenorm_id,
                "sources": [
                    {
                        "resource_id": "infores:diseases",
                        "resource_role": "primary_knowledge_source",
                        "source_record_urls": [row.url]
                    }
                ],
                "attributes": [
                    {
                        "attribute_type_id": "biolink:knowledge_level",
                        "value": "statistical_association"
                    },
                    {
                        "attribute_type_id": "biolink:agent_type",
                        "value": "text_mining_agent"
                    },
                    {  ## no original attribute names since file had no header
                        "attribute_type_id": "biolink:z_score",
                        "value": row.z_score
                    },
                    {
                        "attribute_type_id": "biolink:has_confidence_score",
                        "value": row.confidence_score
                    },
                    {
                        "attribute_type_id": "biolink:original_subject",
                        "value": row.gene_id
                    },
                    {
                        "attribute_type_id": "biolink:original_object",
                        "value": row.disease_id
                    },
                ]
            }
            ## doing so it doesn't print
            bytes = trapi_writer.write(document)
        else:
            ## won't write the document
            print(f"duplicate row encountered: {temp}")

## knowledge data: parser - separate function (diff row abbreviation, document)
    knowledge_tally = set()
    
    ## using itertuples because it's faster, preserves datatypes
    for row in df_knowledge.itertuples(index=False):
        ## construct row abbreviation: needs source_db!
        temp = row.gene_id + "_" + row.disease_id + "_" + row.source_db
        
        if temp not in knowledge_tally:
            knowledge_tally.add(temp)

            document = {
                ## simple assignments: no if
                "subject": row.gene_nodenorm_id,
                "predicate": "biolink:genetically_associated_with",
                "object": row.disease_nodenorm_id,
                "attributes": [
                    {
                        "attribute_type_id": "biolink:knowledge_level",
                        "value": "knowledge_assertion"
                    },
                    {
                        "attribute_type_id": "biolink:agent_type",
                        "value": "manual_agent"
                    },
                    {  ## no original attribute names since file had no header
                        "attribute_type_id": "biolink:has_confidence_score",
                        "value": row.confidence_score
                    },
                    {
                        "attribute_type_id": "biolink:original_subject",
                        "value": row.gene_id
                    },
                    {
                        "attribute_type_id": "biolink:original_object",
                        "value": row.disease_id
                    },
                ]
            }
            ## sources: depends on source_db value
            if row.source_db == "MedlinePlus":
                document["sources"] = [
                    {
                        "resource_id": "infores:medlineplus",
                        "resource_role": "primary_knowledge_source"
                    },
                    {
                        "resource_id": "infores:diseases",
                        "resource_role": "aggregator_knowledge_source",
                        "upstream_resource_ids": ["infores:medlineplus"]
                    }
                ]
            elif row.source_db == "AmyCo":
                document["sources"] = [
                    {
                        "resource_id": "infores:amyco",
                        "resource_role": "primary_knowledge_source"
                    },
                    {
                        "resource_id": "infores:diseases",
                        "resource_role": "aggregator_knowledge_source",
                        "upstream_resource_ids": ["infores:amyco"]
                    }
                ]
            else:
                raise ValueError(f"Unexpected source_db value during source mapping: {row.source_db}. Adjust parser.")
            ## doing so it doesn't print
            bytes = trapi_writer.write(document)
        else:
            ## won't write the document
            print(f"duplicate row encountered: {temp}")

duplicate row encountered: ENSEMBL:ENSP00000269703_DOID:0060655_MedlinePlus
duplicate row encountered: ENSEMBL:ENSP00000272895_DOID:0060655_MedlinePlus
duplicate row encountered: ENSEMBL:ENSP00000311687_DOID:0060655_MedlinePlus
duplicate row encountered: ENSEMBL:ENSP00000342392_DOID:0050568_MedlinePlus
duplicate row encountered: ENSEMBL:ENSP00000379845_DOID:0060407_MedlinePlus


In [79]:
df_knowledge[df_knowledge.duplicated(subset=["gene_id", "disease_id", "source_db"], keep=False)]

,gene_id,gene_name,disease_id,disease_name,source_db,evidence_type,confidence_score,gene_nodenorm_id,gene_nodenorm_label,gene_nodenorm_category,disease_nodenorm_id,disease_nodenorm_label
811,ENSEMBL:ENSP00000269703,CYP4F22,DOID:0060655,Autosomal recessive congenital ichthyosis,MedlinePlus,CURATED,5,NCBIGene:126410,CYP4F22,biolink:Gene,MONDO:0017265,autosomal recessive congenital ichthyosis
812,ENSEMBL:ENSP00000269703,CYP4F22,DOID:0060655,Autosomal recessive congenital ichthyosis,MedlinePlus,CURATED,5,NCBIGene:126410,CYP4F22,biolink:Gene,MONDO:0017265,autosomal recessive congenital ichthyosis
837,ENSEMBL:ENSP00000272895,ABCA12,DOID:0060655,Autosomal recessive congenital ichthyosis,MedlinePlus,CURATED,5,UniProtKB:Q86UK0-1,glucosylceramide transporter ABCA12 isoform h1...,biolink:Protein,MONDO:0017265,autosomal recessive congenital ichthyosis
838,ENSEMBL:ENSP00000272895,ABCA12,DOID:0060655,Autosomal recessive congenital ichthyosis,MedlinePlus,CURATED,5,UniProtKB:Q86UK0-1,glucosylceramide transporter ABCA12 isoform h1...,biolink:Protein,MONDO:0017265,autosomal recessive congenital ichthyosis
1291,ENSEMBL:ENSP00000311687,NIPAL4,DOID:0060655,Autosomal recessive congenital ichthyosis,MedlinePlus,CURATED,5,UniProtKB:Q0D2K0-1,magnesium transporter NIPA4 isoform h1 (human),biolink:Protein,MONDO:0017265,autosomal recessive congenital ichthyosis
1292,ENSEMBL:ENSP00000311687,NIPAL4,DOID:0060655,Autosomal recessive congenital ichthyosis,MedlinePlus,CURATED,5,UniProtKB:Q0D2K0-1,magnesium transporter NIPA4 isoform h1 (human),biolink:Protein,MONDO:0017265,autosomal recessive congenital ichthyosis
1714,ENSEMBL:ENSP00000342392,MESP2,DOID:0050568,Spondylocostal dysostosis,MedlinePlus,CURATED,5,NCBIGene:145873,MESP2,biolink:Gene,MONDO:0000359,spondylocostal dysostosis
1715,ENSEMBL:ENSP00000342392,MESP2,DOID:0050568,Spondylocostal dysostosis,MedlinePlus,CURATED,5,NCBIGene:145873,MESP2,biolink:Gene,MONDO:0000359,spondylocostal dysostosis
2808,ENSEMBL:ENSP00000379845,ABAT,DOID:0060407,Chromosome 18q deletion syndrome,MedlinePlus,CURATED,5,NCBIGene:18,ABAT,biolink:Gene,MONDO:0011147,chromosome 18q deletion syndrome
2809,ENSEMBL:ENSP00000379845,ABAT,DOID:0060407,Chromosome 18q deletion syndrome,MedlinePlus,CURATED,5,NCBIGene:18,ABAT,biolink:Gene,MONDO:0011147,chromosome 18q deletion syndrome


In [80]:
## number of docs that should be created
df_textmining.shape[0] + df_knowledge.shape[0] - 5  ## add together, remove duplicates

269969

### File: KGX edges

In [ ]:
# df_textmining

In [ ]:
## code chunk for testing parts of inner code

# with jsonlines.open('DISEASES_kgx_edges.jsonl', mode='w', compact=True) as kgx_edges_writer:
#     textmining_tally = set()
    
#     ## using itertuples because it's faster, preserves datatypes
#     for row in df_textmining.itertuples(index=False):
#         ## construct row abbreviation
#         temp = row.gene_id + "_" + row.disease_id
        
        
#         if row.gene_id == "ENSEMBL:ENSP00000000412":
#             break    

In [81]:


with jsonlines.open('DISEASES_kgx_edges.jsonl', mode='w', compact=True) as kgx_edges_writer:
    
## text-mined data: parser - separate function (diff row abbreviation, document)
    textmining_tally = set()
    
    ## using itertuples because it's faster, preserves datatypes
    for row in df_textmining.itertuples(index=False):
        ## construct row abbreviation
        temp = row.gene_id + "_" + row.disease_id
        
        if temp not in textmining_tally:
            textmining_tally.add(temp)

            document = {
                "subject": row.gene_nodenorm_id,
                "predicate": "biolink:occurs_together_in_literature_with",
                "object": row.disease_nodenorm_id,
                "sources": [
                    {
                        "resource_id": "infores:diseases",
                        "resource_role": "primary_knowledge_source",
                        "source_record_urls": [row.url]
                    }
                ],
                "knowledge_level": "statistical_association",
                "agent_type": "text_mining_agent",
                "z_score": row.z_score,
                "has_confidence_score": row.confidence_score,
                "original_subject": row.gene_id,
                "original_object": row.disease_id,
            }
            
            ## doing so it doesn't print
            bytes = kgx_edges_writer.write(document)
        else:
            ## won't write the document
            print(f"duplicate row encountered: {temp}")

## knowledge data: parser - separate function (diff row abbreviation, document)
    knowledge_tally = set()
    
    ## using itertuples because it's faster, preserves datatypes
    for row in df_knowledge.itertuples(index=False):
        ## construct row abbreviation: needs source_db!
        temp = row.gene_id + "_" + row.disease_id + "_" + row.source_db
        
        if temp not in knowledge_tally:
            knowledge_tally.add(temp)

            document = {
                ## simple assignments: no if
                "subject": row.gene_nodenorm_id,
                "predicate": "biolink:genetically_associated_with",
                "object": row.disease_nodenorm_id,
                "knowledge_level": "knowledge_assertion",
                "agent_type": "manual_agent",
                "has_confidence_score": row.confidence_score,
                "original_subject": row.gene_id,
                "original_object": row.disease_id,
            }
            
            ## sources: depends on source_db value
            if row.source_db == "MedlinePlus":
                document["sources"] = [
                    {
                        "resource_id": "infores:medlineplus",
                        "resource_role": "primary_knowledge_source"
                    },
                    {
                        "resource_id": "infores:diseases",
                        "resource_role": "aggregator_knowledge_source",
                        "upstream_resource_ids": ["infores:medlineplus"]
                    }
                ]
            elif row.source_db == "AmyCo":
                document["sources"] = [
                    {
                        "resource_id": "infores:amyco",
                        "resource_role": "primary_knowledge_source"
                    },
                    {
                        "resource_id": "infores:diseases",
                        "resource_role": "aggregator_knowledge_source",
                        "upstream_resource_ids": ["infores:amyco"]
                    }
                ]
            else:
                raise ValueError(f"Unexpected source_db value during source mapping: {row.source_db}. Adjust parser.")
            
            ## doing so it doesn't print
            bytes = kgx_edges_writer.write(document)
        else:
            ## won't write the document
            print(f"duplicate row encountered: {temp}")

duplicate row encountered: ENSEMBL:ENSP00000269703_DOID:0060655_MedlinePlus
duplicate row encountered: ENSEMBL:ENSP00000272895_DOID:0060655_MedlinePlus
duplicate row encountered: ENSEMBL:ENSP00000311687_DOID:0060655_MedlinePlus
duplicate row encountered: ENSEMBL:ENSP00000342392_DOID:0050568_MedlinePlus
duplicate row encountered: ENSEMBL:ENSP00000379845_DOID:0060407_MedlinePlus


### File: KGX nodes

Requires id and category. name and other properties (basically node attributes) are optional. 

In [82]:
## get unique list of NodeNormed nodes

## need category 
nodenormed_genes_final = pd.concat([df_textmining[["gene_nodenorm_id", "gene_nodenorm_label", "gene_nodenorm_category"]], 
                         df_knowledge[["gene_nodenorm_id", "gene_nodenorm_label", "gene_nodenorm_category"]]], 
                        ignore_index=True).drop_duplicates()
nodenormed_diseases_final = pd.concat([df_textmining[["disease_nodenorm_id", "disease_nodenorm_label"]], 
                            df_knowledge[["disease_nodenorm_id", "disease_nodenorm_label"]]], 
                           ignore_index=True).drop_duplicates()

nodenormed_genes_final
nodenormed_diseases_final

,gene_nodenorm_id,gene_nodenorm_label,gene_nodenorm_category
0,NCBIGene:381,ARF5,biolink:Gene
3,NCBIGene:4074,M6PR,biolink:Gene
10,NCBIGene:2288,FKBP4,biolink:Gene
34,UniProtKB:Q9NR63-1,cytochrome P450 26B1 isoform 1 (human),biolink:Protein
50,UniProtKB:Q7L592-1,"protein arginine methyltransferase NDUFAF7, mi...",biolink:Protein
...,...,...,...
266008,UniProtKB:Q149M9-3,NACHT domain- and WD repeat-containing protein...,biolink:Protein
266009,UniProtKB:Q86U70-1,LIM domain-binding protein 1 isoform 1 (human),biolink:Protein
266474,UniProtKB:P41219-1,peripherin isoform h1 (human),biolink:Protein
268891,UniProtKB:Q96JH8-4,Ras-associating and dilute domain-containing p...,biolink:Protein


,disease_nodenorm_id,disease_nodenorm_label
0,MONDO:0009271,geroderma osteodysplastica
1,MONDO:0004992,cancer
2,MONDO:0005071,nervous system disorder
3,MONDO:0009650,Mucolipidosis 2
4,MONDO:0018931,"mucolipidosis type III, alpha/beta"
...,...,...
269727,MONDO:0011152,PHGDH deficiency
269742,MONDO:0100428,progressive bulbar palsy of childhood
269753,MONDO:0012791,"mitochondrial DNA depletion syndrome, encephal..."
269951,MONDO:0009949,pyruvate carboxylase deficiency disease


In [83]:


with jsonlines.open('DISEASES_kgx_nodes.jsonl', mode='w', compact=True) as kgx_nodes_writer:
    
    ## using itertuples because it's faster, preserves datatypes
    for row in nodenormed_genes_final.itertuples(index=False):
        ## doing so it doesn't print
        bytes = kgx_nodes_writer.write({
            "id": row.gene_nodenorm_id,
            "name": row.gene_nodenorm_label,
            "category": [row.gene_nodenorm_category]
        })

    ## using itertuples because it's faster, preserves datatypes
    for row in nodenormed_diseases_final.itertuples(index=False):
        ## doing so it doesn't print
        bytes = kgx_nodes_writer.write({
            "id": row.disease_nodenorm_id,
            "name": row.disease_nodenorm_label,
            ## hard-coded because during pre-NodeNorm process, only kept entities with this primary category
            "category": ["biolink:Disease"]
        })

In [84]:
nodenormed_genes_final.shape[0] + nodenormed_diseases_final.shape[0]

21962

## Notes

* __will create edges that look like duplicates because triple is the same, but the original IDs/data is diff. Just leaving it this way for now -> but will need to consider what to do (possible to merge??).__
* confidence_score is sometimes float (from text-mining) and sometimes int (from knowledge). Not worried about, for now. 